<a href="https://colab.research.google.com/github/HNXJ/PredictiveCodingNetwork/blob/main/PredictiveLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np


# GPU config if needed
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Recurrent neural network based on predictive coding properties

In this simulation, we will use task of [Abdullahi et. al 2020] by increasing temporal resolution from one image per step to 10 iterations for each input.

In [114]:
# TB
# dir(tf.keras.layers.LSTM)

In [115]:
def create_serial_dataset(x=None, y=None, n=100, length=10, frames=10):

    X = np.zeros([n, length*frames, x.shape[1], x.shape[2]])
    Y = np.zeros([n, length*frames, 10])

    for i in range(n):

        k = np.random.randint(0, 1000, size=(length))
        for j in range(length):
          
            X[i, j*frames:j*frames+frames, :, :] = x[k[j], :, :]
            Y[i, j*frames:j*frames+frames, y[k[j]]] = 1
          
    return X.reshape(n, length*frames, x.shape[1] * x.shape[2]), Y

def init_MNIST():

    (x_train,y_train), (x_test,y_test) = tf.keras.datasets.mnist.load_data()
    Xn, Yn = create_serial_dataset(x_train, y_train, n=100, length=10, frames=10)
    Xn /= 255.0
    Yn /= 255.0
    X = Xn[:10, :, :]
    Y = Yn[:10, :, :]
    return X, Y, Xn, Yn

In [116]:
class RNNModel1(tf.keras.Model):
    def __init__(self):
        super(RNNModel1, self).__init__()
        self.Input = (tf.keras.layers.InputLayer(input_shape=(None, 784)))
        self.LSTM = tf.keras.layers.LSTM(input_shape=(None, 784),
          units=512,
          recurrent_dropout=0.2,
          return_sequences=True,
          # return_state=True
        )
        self.FCN = tf.keras.layers.Dense(units=10)
        return 
        
    def call(self, x):
        out = self.Input(x)
        out = self.LSTM(x)
        out = self.FCN(out)
        return out

    def get_state(self):
        return

In [117]:
class PredictiveNet():
    def __init__(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.InputLayer(input_shape=(None, 784)))
        self.model.add(tf.keras.layers.LSTM(
          units=512,
          recurrent_dropout=0.2,
          return_sequences=True,
          # return_state=True
        ))
        self.model.add(tf.keras.layers.Dense(units=10))
        self.lastPreactivation = None
        return

    def printw(self): # Debug log
        print(K.mean(self.model.layers[1].weights[0]))
        return

    def getPreactivation(self, x):
        MTemp = K.function([self.model.layers[0].input],
                                  [self.model.layers[1].input])
        stateVector = MTemp(x)
        self.lastPreactivation = stateVector[0]
        return self.lastPreactivation

    def EnergyCostLoss(self, y_true, y_pred):
        error = y_pred - y_true
        lambda1 = 1.2
        lambda2 = 0.0
        lambda3 = 0.0
        # preact = self.lastPreactivation
        return K.mean(K.square(error) + lambda1*K.mean(K.abs(y_pred))) + lambda2*K.mean(K.abs(self.model.layers[0].weights[0])) + lambda3*K.mean(K.abs(self.model.layers[1].weights[0]))

In [118]:
X, Y, Xn, Yn = init_MNIST()
Net1 = PredictiveNet()
# a = model.predict(Xn[1:2, :, :])
Net1.model.compile(
  loss=Net1.EnergyCostLoss,
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)
Net1.model.summary()

Net1.printw()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, None, 512)         2656256   
_________________________________________________________________
dense_7 (Dense)              (None, None, 10)          5130      
Total params: 2,661,386
Trainable params: 2,661,386
Non-trainable params: 0
_________________________________________________________________
tf.Tensor(0.0002055495, shape=(), dtype=float32)


In [119]:
means1 = []
means1a = []

for k in range(10):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(l_pred)))
    means1a.append(np.mean(np.mean(np.abs(l_pred))))

In [120]:
means2 = []
means2a = []

for k in range(10, 30):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
    #     plt.plot(l_pred[0, i, :])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(l_pred)))
    means2a.append(np.mean(np.mean(np.abs(l_pred))))

In [121]:
print("Mean energy on recurrent preactivation (LSTM) on sigmoid activation function before training")
print(np.mean(means1), np.mean(means2), "(ME)")
print(np.mean(means1a), np.mean(means2a), "(MAE)")
print(100*np.abs(np.mean(means1)-np.mean(means2))/np.mean(means1), "(Difference ratio, percentage %)")
print(100*np.abs(np.mean(means1a)-np.mean(means2a))/np.mean(means1a), "(Difference abs ratio, percentage %)")

Mean energy on recurrent preactivation (LSTM) on sigmoid activation function before training
0.00097268086 0.0016234837 (ME)
0.15223369 0.14708312 (MAE)
66.90815910686256 (Difference ratio, percentage %)
3.3833322076734764 (Difference abs ratio, percentage %)


In [122]:
history = Net1.model.fit(
    x=X, y=Y,
    epochs=100,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=True
)

Epoch 1/100
2/2 [==============================] - 3s 333ms/step - loss: 1.1265
Epoch 2/100
2/2 [==============================] - 1s 300ms/step - loss: 0.4551
Epoch 3/100
2/2 [==============================] - 1s 307ms/step - loss: 0.3260
Epoch 4/100
2/2 [==============================] - 1s 312ms/step - loss: 0.2924
Epoch 5/100
2/2 [==============================] - 1s 306ms/step - loss: 0.1181
Epoch 6/100
2/2 [==============================] - 1s 302ms/step - loss: 0.0895
Epoch 7/100
2/2 [==============================] - 1s 301ms/step - loss: 0.1078
Epoch 8/100
2/2 [==============================] - 1s 303ms/step - loss: 0.0888
Epoch 9/100
2/2 [==============================] - 1s 305ms/step - loss: 0.0518
Epoch 10/100
2/2 [==============================] - 1s 301ms/step - loss: 0.0386
Epoch 11/100
2/2 [==============================] - 1s 302ms/step - loss: 0.0485
Epoch 12/100
2/2 [==============================] - 1s 304ms/step - loss: 0.0444
Epoch 13/100
2/2 [===================

In [123]:
# Single epoch for test
history = Net1.model.fit(
    x=X, y=Y,
    epochs=1,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=True
)
# history2 = L1.fit(x=X, y=Y, epochs=1, batch_size=5, validation_split=0.0, verbose=1, shuffle=True)

2/2 [==============================] - 1s 344ms/step - loss: 0.0051


In [124]:
Net1.getPreactivation(Xn[8:9, :, :]).shape

(1, 100, 512)

In [125]:
means1 = []
means1a = []

for k in range(10):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(l_pred)))
    means1a.append(np.mean(np.mean(np.abs(l_pred))))

In [126]:
means2 = []
means2a = []

for k in range(10, 30):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
    #     plt.plot(l_pred[0, i, :])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(l_pred)))
    means2a.append(np.mean(np.mean(np.abs(l_pred))))

In [127]:
print("Mean energy on recurrent preactivation (LSTM) on sigmoid activation function after training")
print(np.mean(means1), np.mean(means2), "(ME)")
print(np.mean(means1a), np.mean(means2a), "(MAE)")
print(100*np.abs(np.mean(means1)-np.mean(means2))/np.mean(means1), "(Difference ratio, percentage %)")
print(100*np.abs(np.mean(means1a)-np.mean(means2a))/np.mean(means1a), "(Difference abs ratio, percentage %)")

Mean energy on recurrent preactivation (LSTM) on sigmoid activation function after training
0.0004226238 0.0005489433 (ME)
0.021469388 0.023349274 (MAE)
29.889349004076305 (Difference ratio, percentage %)
8.756121874427938 (Difference abs ratio, percentage %)


## Lesioning the network

In this section, we've lesioned the RNN via adding a gaussian noise with mean=0 and STD=0.1 to all weights of the network as a simulation of simple lesioning.


In [128]:
Net2 = Net1
w = Net2.model.get_weights()
for i in range(2, len(w)):
    w[i] += (np.random.randn(*w[i].shape))*(np.std(w[i])*2)
Net2.model.set_weights(w)

In [129]:
# Single epoch for test
history = Net2.model.fit(
    x=X, y=Y,
    epochs=1,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=True
)
# history2 = L1.fit(x=X, y=Y, epochs=1, batch_size=5, validation_split=0.0, verbose=1, shuffle=True)

2/2 [==============================] - 1s 309ms/step - loss: 0.2565


In [130]:
means1 = []
means1a = []

for k in range(10):
    l, r = k, k+1
    l_pred = Net2.getPreactivation(Xn[l:r, :, :])
    y_pred = Net2.model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(l_pred)))
    means1a.append(np.mean(np.mean(np.abs(l_pred))))

In [131]:
means2 = []
means2a = []

for k in range(10, 30):
    l, r = k, k+1
    l_pred = Net2.getPreactivation(Xn[l:r, :, :])
    y_pred = Net2.model.predict(Xn[l:r, :, :])

    # for i in range(7):
    #     plt.plot(l_pred[0, i, :])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(l_pred)))
    means2a.append(np.mean(np.mean(np.abs(l_pred))))

In [132]:
print("Mean energy on recurrent preactivation (LSTM) on sigmoid activation function after lesioning")
print(np.mean(means1), np.mean(means2), "(ME)")
print(np.mean(means1a), np.mean(means2a), "(MAE)")
print(100*np.abs(np.mean(means1)-np.mean(means2))/np.mean(means1), "(Difference ratio, percentage %)")
print(100*np.abs(np.mean(means1a)-np.mean(means2a))/np.mean(means1a), "(Difference abs ratio, percentage %)")

Mean energy on recurrent preactivation (LSTM) on sigmoid activation function after lesioning
0.0025135516 0.0030835296 (ME)
0.07883092 0.08794949 (MAE)
22.6762015194678 (Difference ratio, percentage %)
11.567252873609933 (Difference abs ratio, percentage %)
